In [42]:
# from imageProcessing import cut_tif_into_patches_with_hard_boundary

from mmseg.apis import inference_model, init_model, show_result_pyplot
import mmcv
import cv2
import matplotlib.pyplot as plt
import os

import os
from PIL import Image
from tqdm import tqdm

import numpy as np

import time

# from eval_func import eval_func

In [22]:
import sys
sys.path.append('../mask_evaluation')
from main import print_metrics, evaluate_files, evaluate_file

# Functions

In [23]:
# FUNCTIONS

def eval_func(PIMG_PATH, PANN_PATH, PPRED_PATH, model):
     
     try:
        
        PIMG_files = os.listdir(PIMG_PATH)
        PIMG_files = list(filter(lambda x: x.endswith(".tif"), PIMG_files))
        PANN_files = os.listdir(PANN_PATH)
        PANN_files = list(filter(lambda x: x.endswith(".png"), PANN_files))
        PPRED_files = os.listdir(f'{PPRED_PATH}')
        PPRED_files = list(filter(lambda x: x.endswith(".tif"), PPRED_files))

        if len(PPRED_files) == 0:
            __pred_into_files(PIMG_PATH=PIMG_PATH, PIMG_files=PIMG_files, PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, model=model)
            PPRED_files = os.listdir(f'{PPRED_PATH}')
            PPRED_files = list(filter(lambda x: x.endswith(".tif"), PPRED_files))

     except:
          print()

def __extract_green(image):
    img = image
    img[:,:,1] = 255
    img[:,:,2] = 255
    # print(g[0,0,:])

    for i in range(len(img[:,0,0])):
        for ii in range(len(img[0,:,0])):
            if img[i, ii, 0] != 255:
                img[i, ii, :] = [0,0,0]
    img = cv2.bitwise_not(img)
    return img

def __pred_into_files(PIMG_PATH, PIMG_files, PANN_PATH, PANN_files, PPRED_PATH, model):
    for idx, f in enumerate(tqdm(PIMG_files, desc='Predict')):
        fileIdx = idx
        tIMG = cv2.imread(f'{PIMG_PATH}/{PIMG_files[fileIdx]}')
        tIMG = cv2.cvtColor(tIMG, cv2.COLOR_BGR2RGB)
        tANN = cv2.imread(f'{PANN_PATH}/{PANN_files[fileIdx]}')
        loveDa_r18_im_raw = inference_model(model, tIMG)
        loveDa_r18_raw_result = show_result_pyplot(model, np.full_like(tIMG, [255,255,255]), loveDa_r18_im_raw, with_labels=False)
        # plt.imshow(loveDa_r18_raw_result)

        predicted = __extract_green(loveDa_r18_raw_result)
        cv2.imwrite(f'{PPRED_PATH}/{f}', predicted)

def get_eval(PANN_PATH, PANN_files, PPRED_PATH, PPRED_files):
    masks = []
    for i in range(len(PPRED_files)-1):
            masks.append((f'{PANN_PATH}/{PANN_files[i]}', f'{PPRED_PATH}/{PPRED_files[i]}'))
    evaluation = evaluate_files(masks)
    return evaluation

def get_best(PIMG_files, PANN_PATH, PANN_files, PPRED_PATH, PPRED_files, pshow = False, highestF1 = 1.0):
    bestf1 = 0
    bestmask = ()
    bestev = None
    for i in tqdm(range(len(PPRED_files)-1), desc='Process'):
        try:
            ev = evaluate_file(f'{PANN_PATH}/{PANN_files[i]}', f'{PPRED_PATH}/{PPRED_files[i]}')
            if(ev.F1 > bestf1 and ev.F1 < highestF1):
                bestf1 = ev.F1
                bestmask = (f'{PANN_PATH}/{PANN_files[i]}',f'{PPRED_PATH}/{PIMG_files[i]}')
                bestev = ev
        except:
            print(f'Error in File {PIMG_files[i]}')

    print_metrics(bestev)

    if(pshow == True):
        tPANN = cv2.imread(bestmask[0])
        tPPRED = cv2.imread(bestmask[1])

        plt.figure(figsize=(20, 10))
        plt.subplot(1, 3, 1)
        plt.imshow(tPANN)
        plt.title(f'Annotated Mask\n{bestmask[0]}')

        plt.subplot(1, 3, 2)
        plt.imshow(tPPRED)
        plt.title(f'Predicted Mask\n{bestmask[1]}')

def extract_green(image):
    img = image
    img[:,:,1] = 255
    img[:,:,2] = 255
    # print(g[0,0,:])

    for i in range(len(img[:,0,0])):
        for ii in range(len(img[0,:,0])):
            if img[i, ii, 0] != 255:
                img[i, ii, :] = [0,0,0]
    img = cv2.bitwise_not(img)
    return img

## cut_tif_into_patches_with_hard_boundary()

In [24]:

def cut_tif_into_patches_with_hard_boundary(i, d, patch_size, ext):
    if not os.path.exists(d):
        os.makedirs(d)

    files = [f for f in os.listdir(i) if f.endswith(ext)]
    for file in tqdm(files, desc='Processing images'):
        file_path = os.path.join(i, file)
        img = Image.open(file_path)
        
        width, height = img.size
        x_patches = width // patch_size
        y_patches = height // patch_size
        
        for x in range(x_patches):
            for y in range(y_patches):
                left = x* patch_size
                upper = y * patch_size
                right = (x+1) * patch_size
                lower = (y+1) * patch_size
                
                patch = img.crop((left, upper, right, lower))
                base, suffix = os.path.splitext(file)
                parts = base.rsplit('_', 1)
                patch_filename = f'{parts[0]}_{x}_{y}_{parts[1]}{suffix}'
                patch.save(os.path.join(d, patch_filename))

# cut_tif_into_patches_with_hard_boundary(i=IMG_PATH, d=f'{IMG_PATH}/512', patch_size=512, ext='.tif')
# cut_tif_into_patches_with_hard_boundary(i=ANN_PATH, d=f'{ANN_PATH}/512', patch_size=512, ext='.png')

# Init

In [25]:
IMG_PATH = './data/Munich_bin/img_dir'
ANN_PATH = './data/Munich_bin/ann_dir'
# PREDICTED_PATH = '../data/Munich/2023/prediction/samgeo_batches_v3'

IMG_files = os.listdir(IMG_PATH)
IMG_files = list(filter(lambda x: x.endswith(".tif"), IMG_files))
ANN_files = os.listdir(ANN_PATH)
ANN_files = list(filter(lambda x: x.endswith(".png"), ANN_files))

print(IMG_files)
print(ANN_files)

['32688_5332.tif', '32690_5335.tif', '32691_5334.tif', '32692_5335.tif', '32692_5347.tif']
['32688_5332.png', '32690_5335.png', '32691_5334.png', '32692_5335.png', '32692_5347.png']


In [26]:
PIMG_PATH = f'{IMG_PATH}/512'
PANN_PATH = f'{ANN_PATH}/512'

PIMG_files = os.listdir(PIMG_PATH)
PIMG_files = list(filter(lambda x: x.endswith(".tif"), PIMG_files))
PANN_files = os.listdir(PANN_PATH)
PANN_files = list(filter(lambda x: x.endswith(".png"), PANN_files))
print(len(PANN_files))

80


# DeepLab V3

## LoveDa R18 Testing

In [27]:
loveDa_r18_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512.py"
loveDa_r18_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_loveda_20211104_132800-ce0fa0ca.pth"

loveDa_r18_model = init_model(loveDa_r18_config, loveDa_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_loveda_20211104_132800-ce0fa0ca.pth


In [28]:
# test a single image and show the results
fileIdx = 20
tIMG = cv2.imread(f'{PIMG_PATH}/{PIMG_files[fileIdx]}')
tIMG = cv2.cvtColor(tIMG, cv2.COLOR_BGR2RGB)
tANN = cv2.imread(f'{PANN_PATH}/{PANN_files[fileIdx]}')
loveDa_r18_im_raw = inference_model(loveDa_r18_model, tIMG)
loveDa_r18_raw_result = show_result_pyplot(loveDa_r18_model, np.full_like(tIMG, [255,255,255]), loveDa_r18_im_raw, with_labels=False)
# plt.imshow(loveDa_r18_raw_result)

predicted = extract_green(loveDa_r18_raw_result)
# plt.imshow(predicted)

# plt.figure(figsize=(15, 10))
# plt.subplot(1, 3, 1)
# plt.imshow(tIMG)
# plt.title(f'Geospatial Image\n{PIMG_PATH}/{PIMG_files[fileIdx]}')
# plt.subplot(1, 3, 2)
# plt.imshow(tANN)
# plt.title(f'True Mask\n{PANN_PATH}/{PANN_files[fileIdx]}')
# plt.subplot(1, 3, 3)
# plt.imshow(predicted)
# plt.title(f'Predicted Mask')

# plt.tight_layout()
# plt.show()

In [29]:
DATASET = 'deepLabV3Plus'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{DATASET}'

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

PPRED_files = os.listdir(f'{PPRED_PATH}')
PPRED_files = list(filter(lambda x: x.endswith(".tif"), PPRED_files))
print(len(PPRED_files))

59


In [30]:
for idx, f in enumerate(PIMG_files):
    fileIdx = idx
    tIMG = cv2.imread(f'{PIMG_PATH}/{PIMG_files[fileIdx]}')
    tIMG = cv2.cvtColor(tIMG, cv2.COLOR_BGR2RGB)
    tANN = cv2.imread(f'{PANN_PATH}/{PANN_files[fileIdx]}')
    loveDa_r18_im_raw = inference_model(loveDa_r18_model, tIMG)
    loveDa_r18_raw_result = show_result_pyplot(loveDa_r18_model, np.full_like(tIMG, [255,255,255]), loveDa_r18_im_raw, with_labels=False)
    # plt.imshow(loveDa_r18_raw_result)

    predicted = extract_green(loveDa_r18_raw_result)
    cv2.imwrite(f'{PPRED_PATH}/{f}', predicted)

In [31]:
idxx = fileIdx
print(f'PANN: {PANN_PATH}/{PANN_files[idxx]} -- PPRED: {PPRED_PATH}/{PPRED_files[idxx]}')
ev = evaluate_file(f'{PANN_PATH}/{PANN_files[idxx]}', f'{PPRED_PATH}/{PPRED_files[idxx]}')


tPANN = cv2.imread(f'{PANN_PATH}/{PANN_files[fileIdx]}')
tPPRED = cv2.imread(f'{PPRED_PATH}/{PPRED_files[fileIdx]}')

plt.figure(figsize=(15, 10))
plt.subplot(1, 3, 1)
plt.imshow(tPANN)
plt.title(f'Annotated Mask\n/{PANN_files[fileIdx]}')

plt.subplot(1, 3, 2)
plt.imshow(tPPRED)
plt.title(f'Predicted Mask\n/{PPRED_files[fileIdx]}')

print_metrics(ev)


IndexError: list index out of range

In [ ]:
masks = []
for i in range(len(PPRED_files)-1):
    masks.append((f'{PANN_PATH}/{PANN_files[i]}', f'{PPRED_PATH}/{PPRED_files[i]}'))
evaluation = evaluate_files(masks)
print('\nALL MASKS\n')
print_metrics(evaluation)

In [ ]:
bestf1 = 0
bestmask = ()
bestev = None
for i in tqdm(range(len(PPRED_files)-1), desc='Process'):
    try:
        ev = evaluate_file(f'{PANN_PATH}/{PANN_files[i]}', f'{PPRED_PATH}/{PPRED_files[i]}')
        if(ev.F1 > bestf1 and ev.F1 <0.7):
            bestf1 = ev.F1
            bestmask = (f'{PANN_PATH}/{PANN_files[i]}',f'{PPRED_PATH}/{PIMG_files[i]}')
            bestev = ev
    except:
        print(f'Error in File {PIMG_files[i]}')

print_metrics(bestev)

tPANN = cv2.imread(bestmask[0])
tPPRED = cv2.imread(bestmask[1])

plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
plt.imshow(tPANN)
plt.title(f'Annotated Mask\n{bestmask[0]}')

plt.subplot(1, 3, 2)
plt.imshow(tPPRED)
plt.title(f'Predicted Mask\n{bestmask[1]}')


## LoveDa R18

In [43]:
loveDa_r18_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r18-d8_4xb4-80k_loveda-512x512.py"
loveDa_r18_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_loveda_20211104_132800-ce0fa0ca.pth"

loveDa_r18_model = init_model(loveDa_r18_config, loveDa_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'LoveDa_R18'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

c:\Users\henha\miniconda3\envs\openmmlab\Lib\site-packages\mmseg\models\builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
c:\Users\henha\miniconda3\envs\openmmlab\Lib\site-packages\mmseg\models\losses\cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_loveda_20211104_132800-ce0fa0ca.pth


In [45]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=loveDa_r18_model)
deep_loveda_R18 = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(deep_loveda_R18)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')


Predict:   0%|          | 0/80 [00:00<?, ?it/s]

c:\Users\henha\miniconda3\envs\openmmlab\Lib\site-packages\mmengine\visualization\visualizer.py:196: UserWarning: Failed to add <class 'mmengine.visualization.vis_backend.LocalVisBackend'>, please provide the `save_dir` argument.
  warnings.warn(f'Failed to add {vis_backend.__class__}, '
Predict: 100%|██████████| 80/80 [01:40<00:00,  1.25s/it]


Results: 
   accuracy: 78.766 %
         f1:  0.529
     recall:  0.361 (or sensitivity)
  precision:  0.988
specificity:  0.998
        iou:  0.359 (or Jaccard index)

TPR: 0.361 | FNR: 0.639
TNR: 0.998 | FPR: 0.002
PPV: 0.988 | FDR: 0.012
NPV: 0.760 | FOR: 0.240
Elapsed Time: 100.45s = 1.67m


In [ ]:
get_best(PIMG_files, PANN_PATH, PANN_files, PPRED_PATH, PPRED_files, True, 0.91)

## LoveDa R50

In [38]:
loveDa_r50_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r50-d8_4xb4-80k_loveda-512x512.py"
loveDa_r50_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r50-d8_512x512_80k_loveda_20211105_080442-f0720392.pth"

loveDa_r50_model = init_model(loveDa_r50_config, loveDa_r50_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'LoveDa_R50'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r50-d8_512x512_80k_loveda_20211105_080442-f0720392.pth


In [39]:
if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=loveDa_r50_model)
LoveDa_R50_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(LoveDa_R50_ev)

Results: 
   accuracy: 77.843 %
         f1:  0.496
     recall:  0.331 (or sensitivity)
  precision:  0.993
specificity:  0.999
        iou:  0.330 (or Jaccard index)

TPR: 0.331 | FNR: 0.669
TNR: 0.999 | FPR: 0.001
PPV: 0.993 | FDR: 0.007
NPV: 0.752 | FOR: 0.248


## LoveDa R101

In [36]:
loveDa_r101_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r101-d8_4xb4-80k_loveda-512x512.py"
loveDa_r101_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r101-d8_512x512_80k_loveda_20211105_110759-4c1f297e.pth"

loveDa_r101_model = init_model(loveDa_r101_config, loveDa_r101_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'LoveDa_R101'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r101-d8_512x512_80k_loveda_20211105_110759-4c1f297e.pth


In [37]:
if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=loveDa_r101_model)
LoveDa_R101_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(LoveDa_R101_ev)

Results: 
   accuracy: 78.501 %
         f1:  0.520
     recall:  0.353 (or sensitivity)
  precision:  0.986
specificity:  0.997
        iou:  0.352 (or Jaccard index)

TPR: 0.353 | FNR: 0.647
TNR: 0.997 | FPR: 0.003
PPV: 0.986 | FDR: 0.014
NPV: 0.758 | FOR: 0.242


## Potsdam R18

In [46]:
potsdam_r18_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r18-d8_4xb4-80k_potsdam-512x512.py"
potsdam_r18_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_potsdam_20211219_020601-75fd5bc3.pth"

potsdam_r18_model = init_model(potsdam_r18_config, potsdam_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'Potsdam_R18'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

c:\Users\henha\miniconda3\envs\openmmlab\Lib\site-packages\mmseg\models\builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
c:\Users\henha\miniconda3\envs\openmmlab\Lib\site-packages\mmseg\models\losses\cross_entropy_loss.py:250: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r18-d8_512x512_80k_potsdam_20211219_020601-75fd5bc3.pth


In [47]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=potsdam_r18_model)
Potsdam_R18_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(Potsdam_R18_ev)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')

Predict:   0%|          | 0/80 [00:00<?, ?it/s]

Predict: 100%|██████████| 80/80 [01:40<00:00,  1.25s/it]


Results: 
   accuracy: 47.084 %
         f1:  0.299
     recall:  0.342 (or sensitivity)
  precision:  0.266
specificity:  0.534
        iou:  0.176 (or Jaccard index)

TPR: 0.342 | FNR: 0.658
TNR: 0.534 | FPR: 0.466
PPV: 0.266 | FDR: 0.734
NPV: 0.623 | FOR: 0.377
Elapsed Time: 100.48s = 1.67m


## Vaihingen R18

In [48]:
vaihingen_r18_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r18-d8_4xb4-80k_vaihingen-512x512.py"
vaihingen_r18_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r18-d8_4x4_512x512_80k_vaihingen_20211231_230805-7626a263.pth"

vaihingen_r18_model = init_model(vaihingen_r18_config, vaihingen_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'Vaihingen_R18'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r18-d8_4x4_512x512_80k_vaihingen_20211231_230805-7626a263.pth


In [49]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=vaihingen_r18_model)
Vaihingen_R18_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(Vaihingen_R18_ev)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')

Predict:   0%|          | 0/80 [00:00<?, ?it/s]

Predict: 100%|██████████| 80/80 [01:43<00:00,  1.29s/it]


Results: 
   accuracy: 27.224 %
         f1:  0.376
     recall:  0.664 (or sensitivity)
  precision:  0.262
specificity:  0.079
        iou:  0.231 (or Jaccard index)

TPR: 0.664 | FNR: 0.336
TNR: 0.079 | FPR: 0.921
PPV: 0.262 | FDR: 0.738
NPV: 0.324 | FOR: 0.676
Elapsed Time: 103.64s = 1.73m


## Mapillary Vistas v1.2 R50

In [51]:
mapvis_r50_config = r"mmsegmentation\configs\deeplabv3plus\deeplabv3plus_r50-d8_4xb2-300k_mapillay_v1_65-1280x1280.py"
mapvis_r50_checkpoint = r"checkpoints\DLV3P\deeplabv3plus_r50-d8_4xb2-300k_mapillay_v1_65-1280x1280_20230301_110504-655f8e43.pth"

mapvis_r50_model = init_model(mapvis_r50_config, mapvis_r50_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'deepLabV3Plus'
DATASET = 'MapVis_R50'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\DLV3P\deeplabv3plus_r50-d8_4xb2-300k_mapillay_v1_65-1280x1280_20230301_110504-655f8e43.pth


In [52]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=mapvis_r50_model)
MapVis_R50_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(MapVis_R50_ev)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')

Predict: 100%|██████████| 80/80 [02:58<00:00,  2.23s/it]


Results: 
   accuracy: 33.058 %
         f1:  0.496
     recall:  1.000 (or sensitivity)
  precision:  0.330
specificity:  0.001
        iou:  0.330 (or Jaccard index)

TPR: 1.000 | FNR: 0.000
TNR: 0.001 | FPR: 0.999
PPV: 0.330 | FDR: 0.670
NPV: 0.999 | FOR: 0.001
Elapsed Time: 178.43s = 2.97m


# PSPNet

## LoveDa R18

In [53]:
psp_loveDa_r18_config = r"mmsegmentation\configs\pspnet\pspnet_r18-d8_4xb4-80k_loveda-512x512.py"
psp_loveDa_r18_checkpoint = r"checkpoints\PSPNET\pspnet_r18-d8_512x512_80k_loveda_20211105_052100-b97697f1.pth"

psp_loveDa_r18_model = init_model(psp_loveDa_r18_config, psp_loveDa_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'pspnet'
DATASET = 'LoveDa_R18'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\PSPNET\pspnet_r18-d8_512x512_80k_loveda_20211105_052100-b97697f1.pth


In [54]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=psp_loveDa_r18_model)
psp_loveda_R18_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(psp_loveda_R18_ev)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')

Predict:   0%|          | 0/80 [00:00<?, ?it/s]

Predict: 100%|██████████| 80/80 [01:49<00:00,  1.37s/it]


Results: 
   accuracy: 79.198 %
         f1:  0.544
     recall:  0.376 (or sensitivity)
  precision:  0.984
specificity:  0.997
        iou:  0.373 (or Jaccard index)

TPR: 0.376 | FNR: 0.624
TNR: 0.997 | FPR: 0.003
PPV: 0.984 | FDR: 0.016
NPV: 0.764 | FOR: 0.236
Elapsed Time: 110.37s = 1.84m


# HRNET

## LoveDa R18

In [55]:
hr_loveDa_r18_config = r"mmsegmentation\configs\hrnet\fcn_hr18_4xb4-80k_loveda-512x512.py"
hr_loveDa_r18_checkpoint = r"checkpoints\HRNET\fcn_hr18_512x512_80k_loveda_20211210_203952-93d9c3b3.pth"

hr_loveDa_r18_model = init_model(hr_loveDa_r18_config, hr_loveDa_r18_checkpoint,cfg_options=dict(classes=('forest'),palette=[[0, 255, 0]]))

NET = 'hrnet'
DATASET = 'LoveDa_R18'
PPRED_PATH = f'./data/Munich_bin/pred_dir/{NET}/{DATASET}'

Loads checkpoint by local backend from path: checkpoints\HRNET\fcn_hr18_512x512_80k_loveda_20211210_203952-93d9c3b3.pth


In [56]:
s = time.time()

if not os.path.exists(PPRED_PATH):
        os.makedirs(PPRED_PATH)

eval_func(PIMG_PATH=PIMG_PATH, PANN_PATH=PANN_PATH, PPRED_PATH=PPRED_PATH, model=hr_loveDa_r18_model)
hr_loveda_R18_ev = get_eval(PANN_PATH=PANN_PATH, PANN_files=PANN_files, PPRED_PATH=PPRED_PATH, PPRED_files=PPRED_files)
print_metrics(hr_loveda_R18_ev)

e = time.time()
t = e-s
print(f'Elapsed Time: {t:.2f}s = {t/60:.2f}m')

Predict:   0%|          | 0/80 [00:00<?, ?it/s]

Predict: 100%|██████████| 80/80 [01:28<00:00,  1.11s/it]


Results: 
   accuracy: 78.711 %
         f1:  0.527
     recall:  0.360 (or sensitivity)
  precision:  0.985
specificity:  0.997
        iou:  0.358 (or Jaccard index)

TPR: 0.360 | FNR: 0.640
TNR: 0.997 | FPR: 0.003
PPV: 0.985 | FDR: 0.015
NPV: 0.760 | FOR: 0.240
Elapsed Time: 88.94s = 1.48m
